# 데이터 수집

## 카카오 API : 키워드 검색

- 카카오 API 테스트
  - 앱 ID : 
  - 네이티브 앱 키 : 
  - REST API 키 : 
  - JavaScript 키 : 
  - Admin 키 : 

- 키워드 검색
  - 질의어를 통해 장소 검색 결과 반환
  - category_group_name, category_name, place_url
- 키워드 장소 검색 : 일간 100,000건
- 검색 키워드 선정 :
  - '맛집', '분위기 좋은', '테마파크' ,'오션뷰', '감성', '가족여행', '체험', '휴식', '레포츠', '가볼만한 곳'
- 검색할 카테고리 선정 : 숙박을 제외하고 검색 진행
  - CT1(문화시설), AT4(관광명소), FD6(음식점), CE7(카페), AD5(숙박)
- 법정동·리 별로 검색 :<a href = 'https://ko.wikipedia.org/wiki/%EC%A0%9C%EC%A3%BC%EC%8B%9C%EC%9D%98_%ED%96%89%EC%A0%95_%EA%B5%AC%EC%97%AD' target='_blink'>제주시 위키백과</a> , <a href = "https://ko.wikipedia.org/wiki/%EC%84%9C%EA%B7%80%ED%8F%AC%EC%8B%9C%EC%9D%98_%ED%96%89%EC%A0%95_%EA%B5%AC%EC%97%AD" target="_blink">서귀포시 위키백과</a>

### 제주시

#### 1. 카카오 API 카테고리와 직접 선정 키워드 활용

In [1]:
# 제주시
import pandas as pd

jeju_range = pd.read_excel('제주도_제주시_법정동_리.xlsx')
jeju_range.head()

,이름,한자,인구,면적,법정동_리
0,애월읍,涯月邑,31790,202.16,"고내리,고성리,곽지리,광령리,구엄리,금성리,남읍리,봉성리,상가리,상귀리,소길리,수산..."
1,조천읍,朝天邑,21547,150.64,"교래리,대흘리,북촌리,선흘리,신촌리,신흥리,와산리,와흘리,조천리,함덕리"
2,한림읍,翰林邑,23143,91.09,"귀덕리,금능리,금악리,대림리,동명리,명월리,상대리,상명리,수원리,옹포리,월령리,월림..."
3,구좌읍,舊左邑,15216,185.93,"김녕리,덕천리,동복리,상도리,세화리,송당리,월정리,종달리,평대리,하도리,한동리,행원리"
4,한경면,翰京面,8775,79.03,"고산리,금등리,낙천리,두모리,신창리,용수리,저지리,조수리,청수리,판포리"


##### ◽카테고리, 키워드, 지역 변수

In [3]:
keywords = ['맛집', '분위기 좋은', '테마파크' ,'오션뷰', '감성', '가족여행', '체험', '휴식', '레포츠', '가볼만한 곳']
categorys = ['CT1', 'AT4', 'FD6', 'CE7']
categorys_info = {'CT1' : '문화시설', 'AT4' : '관광명소', 'FD6' : '음식점', 'CE7' : '카페'}

In [4]:
# 법정동_리를 split을 활용하여 나눔 example
jeju_range['법정동_리'][0].split(',')[:5]

['고내리', '고성리', '곽지리', '광령리', '구엄리']

#### ◽카카오 API(키워드 검색) JSON 형식
  - 'documents' : ['address_name', 'category_group_code', 'category_group_name', 'category_name', 'distance', 'id', 'phone', 'place_name', 'place_url', 'road_address_name', 'x', 'y']
  - 'meta' : ['is_end', 'pageable_count', 'same_name', 'total_count']

##### ◽카카오 API 활용 함수
- (키워드, 카테고리, 법정동_리) 검색 함수

In [ ]:
import json
import requests

def search_result(keyword, category, jeju_name):
    result = []

    # REST 키
    rest_api_key = ''
    # 헤더
    headers = {"Authorization" : "KakaoAK {}".format(rest_api_key)}
    # 파라미터
    params = {"query" : f"제주특별자치도 {jeju_name} {keyword}", "page" : 1, "category_group_code" : f"{category}"}
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"

    while True:
        # GET을 이용하여 획득
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            # Json을 이용하여 해제
            doc = json.loads(res.text)
            result.extend(doc['documents'])
            if doc['meta']['is_end'] == True:
                break
            else:
                params['page'] += 1
    return result

- 전체 결과 데이터 프레임 반환 함수

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

def search_df():
    results = []
    for idx, row in tqdm(jeju_range.iterrows()):
        for jeju in row['법정동_리'].split(','):
            for category in categorys:
                for key in keywords:
                    r = pd.DataFrame(search_result(key, category, jeju))
                    r['keyword'] = key
                    results.append(r.copy())
    return pd.concat(results).reset_index(drop=True)jeju_poi.head(2)

- 카카오 API 활용 데이터 획득

In [ ]:
jeju_poi = search_df()
jeju_poi.to_excel('./data/220114/제주도_POI(API).xlsx',index=False)

In [ ]:
jeju_poi.head(2)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,테마파크,제주특별자치도 제주시 연동 1320,CT1,문화시설,"문화,예술 > 문화시설 > 박물관",,26388484,064-742-3700,수목원테마파크 아이스뮤지엄,http://place.map.kakao.com/26388484,제주특별자치도 제주시 은수길 69,126.488397743899,33.4707773213401
1,테마파크,제주특별자치도 제주시 애월읍 신엄리 2880-12,AT4,관광명소,"여행 > 관광,명소 > 테마파크",,1129394481,064-748-4245,고스트타운,http://place.map.kakao.com/1129394481,제주특별자치도 제주시 애월읍 부룡수길 35-14,126.35693587972,33.4761948388957


#### ◽데이터 확인(제주도_POI(API))
- 기본 정보 확인
  - NaN 데이터는 존재하지 않는다.
- distance 컬럼은 값이 존재하지 않음
- 엑셀을 통해 중복(id) 제거

In [ ]:
import pandas as pd

api_poi = pd.read_excel('./data/220114/제주도_POI(API)3.xlsx', index_col=False)
api_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5943 entries, 0 to 5942
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              5943 non-null   object 
 1   address_name         5943 non-null   object 
 2   category_group_code  5943 non-null   object 
 3   category_group_name  5943 non-null   object 
 4   category_name        5943 non-null   object 
 5   distance             0 non-null      float64
 6   id                   5943 non-null   int64  
 7   phone                3915 non-null   object 
 8   place_name           5943 non-null   object 
 9   place_url            5943 non-null   object 
 10  road_address_name    5224 non-null   object 
 11  x                    5943 non-null   float64
 12  y                    5943 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 603.7+ KB


In [ ]:
pd.DataFrame(api_poi['keyword'].value_counts())

,keyword
맛집,4768
가볼만한 곳,802
분위기 좋은,260
테마파크,59
오션뷰,22
감성,19
체험,5
가족여행,4
휴식,2
레포츠,2


In [ ]:
pd.DataFrame(api_poi['category_group_name'].value_counts())

,category_group_name
음식점,3028
카페,2048
관광명소,855
문화시설,12


#### ◽id 중복 처리 : 키워드 합치기
- id 중복 확인
- id 값으로 조회하여 값이 2개이상인 경우
  - 각 데이터의 키워드를 합치고 1개의 행만 남긴다.
  - included : 중복된 id일 경우를 식별하기 위해 dict형으로 확인한 경우 추가
  - del_index : 중복된 id의 인덱스 중 1개만 사용할 것이기에 삭제할 인덱스 추가

In [ ]:
import pandas as pd

api_poi = pd.read_excel('./data/220114/제주도_POI(API)3.xlsx', index_col=False)

In [ ]:
included = {}
del_index = []
for idx, row in api_poi.iterrows():
    id = row['id']
    temp = api_poi[api_poi['id'] == id].copy()
    cnt = len(temp)
    if id not in included and cnt > 1:
        included[id] = True
        for i, r in temp.iterrows():
            if idx == i:
                continue
            else:
                api_poi.loc[idx, 'keyword'] = api_poi.loc[idx, 'keyword'] + ',' + r['keyword']
                del_index.append(i)

In [ ]:
# 중복 id의 수, 삭제할 인덱스의 수
len(included.keys()), len(del_index)

(194, 197)

In [ ]:
# 중복 id의 인덱스 삭제
api_poi_del = api_poi.drop(del_index, axis=0)

In [ ]:
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5746 entries, 0 to 5942
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              5746 non-null   object 
 1   address_name         5746 non-null   object 
 2   category_group_code  5746 non-null   object 
 3   category_group_name  5746 non-null   object 
 4   category_name        5746 non-null   object 
 5   distance             0 non-null      float64
 6   id                   5746 non-null   int64  
 7   phone                3765 non-null   object 
 8   place_name           5746 non-null   object 
 9   place_url            5746 non-null   object 
 10  road_address_name    5030 non-null   object 
 11  x                    5746 non-null   float64
 12  y                    5746 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 628.5+ KB


In [ ]:
api_poi_del.to_excel('./data/220114/제주도_POI(API)4.xlsx', index=False)

In [ ]:
# id 중복 제거 데이터 확인(제주도_POI(API)4)
import pandas as pd

api_poi_del = pd.read_excel('./data/220114/제주도_POI(API)4.xlsx', index_col=False)
api_poi_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5746 entries, 0 to 5745
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              5746 non-null   object 
 1   address_name         5746 non-null   object 
 2   category_group_code  5746 non-null   object 
 3   category_group_name  5746 non-null   object 
 4   category_name        5746 non-null   object 
 5   distance             0 non-null      float64
 6   id                   5746 non-null   int64  
 7   phone                3765 non-null   object 
 8   place_name           5746 non-null   object 
 9   place_url            5746 non-null   object 
 10  road_address_name    5030 non-null   object 
 11  x                    5746 non-null   float64
 12  y                    5746 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 583.7+ KB


- 행의 수와 id의 수가 일치하므로 중복된 id가 없음을 확인할 수 있다.

In [ ]:
len(api_poi_del['id'].unique())

5746

In [ ]:
pd.DataFrame(api_poi_del['category_group_name'].value_counts())

,category_group_name
음식점,2969
카페,1950
관광명소,815
문화시설,12


### 서귀포시

- 수집된 데이터를 확인한 결과 '서귀포' 데이터가 없는 것을 확인
- 서귀포에 대한 부분만 새로 획득

#### 1. 카카오 API 카테고리와 직접 선정 키워드 활용

##### ◽카테고리, 키워드, 지역 변수

In [1]:
keywords = ['맛집', '분위기 좋은', '테마파크' ,'오션뷰', '감성', '가족여행', '체험', '휴식', '레포츠', '가볼만한 곳']
categorys = ['CT1', 'AT4', 'FD6', 'CE7']
categorys_info = {'CT1' : '문화시설', 'AT4' : '관광명소', 'FD6' : '음식점', 'CE7' : '카페'}
jeju_range = ['법환동', '서호동', '호근동', '강정동', '도순동', '영남동', '월평동', '동홍동', '서홍동', 
            '보목동','서귀동','토평동', '상효동', '상예동', '색달동', '하예동', '대포동', '중문동', '하원동',
             '회수동', '신효동', '하효동']

##### ◽카카오 API 활용 함수
- (키워드, 카테고리, 법정동_리) 검색 함수
- search_result(keyword, category, jeju_name)
  - (카테고리, 법정동_리) 검색 함수

In [ ]:
import json
import requests

def search_result(keyword, category, jeju_name):
    result = []

    # REST 키
    rest_api_key = 'API Key'
    # 헤더
    headers = {"Authorization" : "KakaoAK {}".format(rest_api_key)}
    # 파라미터
    params = {"query" : f"제주특별자치도 {jeju_name} {keyword}", "page" : 1, "category_group_code" : f"{category}"}
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"

    while True:
        # GET을 이용하여 획득
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            # Json을 이용하여 해제
            doc = json.loads(res.text)
            result.extend(doc['documents'])
            if doc['meta']['is_end'] == True:
                break
            else:
                params['page'] += 1
    return result

- 전체 결과 데이터 프레임 반환 함수

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

def search_df():
    results = []
    for idx, jeju in tqdm(enumerate(jeju_range)):
        for category in categorys:
            for key in keywords:
                r = pd.DataFrame(search_result(key, category, jeju))
                r['keyword'] = key
                results.append(r.copy())
    return pd.concat(results).reset_index(drop=True)

- 카카오 API 활용 데이터 획득

In [ ]:
jeju_poi_additional = search_df()
jeju_poi_additional.to_excel('./data/220119/제주_POI_서귀포.xlsx',index=False)
jeju_poi_additional.head(2)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,테마파크,제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874293757,33.2470161809819
1,테마파크,제주특별자치도 서귀포시 법환동 914,AT4,관광명소,"여행 > 관광,명소 > 테마파크 > 워터테마파크",,17150892,064-739-1930,제주워터월드,http://place.map.kakao.com/17150892,제주특별자치도 서귀포시 월드컵로 33,126.50854558896376,33.24550727132407


#### ◽데이터 확인(제주도_서귀포시_POI(API))
- 기본 정보 확인
  - NaN 데이터는 존재하지 않는다.
- distance 컬럼은 값이 존재하지 않음
- 엑셀을 통해 중복(id) 제거

In [ ]:
import pandas as pd

jeju_poi_additional = pd.read_excel('./data/220119/제주_POI_서귀포.xlsx', index_col=False)
jeju_poi_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4157 entries, 0 to 4156
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              4157 non-null   object 
 1   address_name         4157 non-null   object 
 2   category_group_code  4157 non-null   object 
 3   category_group_name  4157 non-null   object 
 4   category_name        4157 non-null   object 
 5   distance             0 non-null      float64
 6   id                   4157 non-null   int64  
 7   phone                2510 non-null   object 
 8   place_name           4157 non-null   object 
 9   place_url            4157 non-null   object 
 10  road_address_name    3461 non-null   object 
 11  x                    4157 non-null   float64
 12  y                    4157 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 422.3+ KB


In [ ]:
pd.DataFrame(jeju_poi_additional['keyword'].value_counts())

,keyword
맛집,1980
가볼만한 곳,990
분위기 좋은,843
테마파크,175
오션뷰,76
감성,56
가족여행,25
체험,10
레포츠,2


#### ◽id 중복 처리 : 키워드 합치기
- id 중복 확인
- id 값으로 조회하여 값이 2개이상인 경우
  - 각 데이터의 키워드를 합치고 1개의 행만 남긴다.
  - included : 중복된 id일 경우를 식별하기 위해 dict형으로 확인한 경우 추가
  - del_index : 중복된 id의 인덱스 중 1개만 사용할 것이기에 삭제할 인덱스 추가

In [ ]:
# id 중복 확인
import pandas as pd

jeju_poi_additional = pd.read_excel('./data/220119/제주_POI_서귀포.xlsx', index_col=False)
len(jeju_poi_additional), len(jeju_poi_additional['id'].unique())

(4157, 1708)

In [ ]:
jeju_poi_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4157 entries, 0 to 4156
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              4157 non-null   object 
 1   address_name         4157 non-null   object 
 2   category_group_code  4157 non-null   object 
 3   category_group_name  4157 non-null   object 
 4   category_name        4157 non-null   object 
 5   distance             0 non-null      float64
 6   id                   4157 non-null   int64  
 7   phone                2510 non-null   object 
 8   place_name           4157 non-null   object 
 9   place_url            4157 non-null   object 
 10  road_address_name    3461 non-null   object 
 11  x                    4157 non-null   float64
 12  y                    4157 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 422.3+ KB


In [ ]:
#id 중복 처리 : 키워드 합치기
import pandas as pd

jeju_poi_additional = pd.read_excel('./data/220119/제주_POI_서귀포.xlsx', index_col=False)
jeju_poi_additional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4157 entries, 0 to 4156
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              4157 non-null   object 
 1   address_name         4157 non-null   object 
 2   category_group_code  4157 non-null   object 
 3   category_group_name  4157 non-null   object 
 4   category_name        4157 non-null   object 
 5   distance             0 non-null      float64
 6   id                   4157 non-null   int64  
 7   phone                2510 non-null   object 
 8   place_name           4157 non-null   object 
 9   place_url            4157 non-null   object 
 10  road_address_name    3461 non-null   object 
 11  x                    4157 non-null   float64
 12  y                    4157 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 422.3+ KB


In [ ]:
included = {}
del_index = []
for idx, row in jeju_poi_additional.iterrows():
    id = row['id']
    temp = jeju_poi_additional[jeju_poi_additional['id'] == id].copy()
    cnt = len(temp)
    if id not in included and cnt > 1:
        included[id] = True
        for i, r in temp.iterrows():
            if idx == i:
                continue
            else:
                if r['keyword'] not in jeju_poi_additional.loc[idx, 'keyword']:
                    jeju_poi_additional.loc[idx, 'keyword'] = jeju_poi_additional.loc[idx, 'keyword'] + ',' + r['keyword']
                del_index.append(i)

In [ ]:
# 중복 id의 수, 삭제할 인덱스의 수
len(included.keys()), len(del_index)

(652, 2449)

In [ ]:
# 중복 id의 인덱스 삭제
jeju_poi_additional_del = jeju_poi_additional.drop(del_index, axis=0)

- 행의 수와 id의 수가 일치하므로 중복된 id가 없음을 확인할 수 있다.

In [ ]:
len(jeju_poi_additional_del['id'].unique()), len(jeju_poi_additional_del)

(1708, 1708)

In [ ]:
jeju_poi_additional_del.to_excel('./data/220119/제주_POI_서귀포_키워드묶음.xlsx', index=False)

In [ ]:
# id 중복 제거 데이터 확인
import pandas as pd

jeju_poi_add = pd.read_excel('./data/220119/제주_POI_서귀포_키워드묶음.xlsx', index_col=False)
jeju_poi_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1708 non-null   object 
 1   address_name         1708 non-null   object 
 2   category_group_code  1708 non-null   object 
 3   category_group_name  1708 non-null   object 
 4   category_name        1708 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1708 non-null   int64  
 7   phone                1143 non-null   object 
 8   place_name           1708 non-null   object 
 9   place_url            1708 non-null   object 
 10  road_address_name    1498 non-null   object 
 11  x                    1708 non-null   float64
 12  y                    1708 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 173.6+ KB


In [ ]:
pd.DataFrame(jeju_poi_add['category_group_name'].value_counts())

,category_group_name
음식점,866
카페,559
관광명소,279
문화시설,4


### 제주시 & 서귀포시 크롤링 데이터 확인

#### ◽데이터 확인

In [ ]:
import pandas as pd

jeju_poi_add = pd.read_excel('./data/220119/제주_POI_서귀포_키워드묶음.xlsx', index_col=False)
jeju_poi = pd.read_excel('./data/220119/_제주도_POI_컬럼 정리.xlsx', index_col = 0)

- 제주시 데이터

In [ ]:
jeju_poi.head(1)

,keyword,address_name,category_group_name,category_name,id,place_name,x,y,rating
0,테마파크,제주특별자치도 제주시 연동 1320,관광명소,"문화,예술 > 문화시설 > 박물관",26388484,수목원테마파크 아이스뮤지엄,126.488398,33.470777,1.0


- 서귀포시 데이터

In [ ]:
jeju_poi_add.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874,33.247016


- 제주시 & 서귀포시 중복 데이터 제거
    - id를 기준으로 jeju_poi에 수집된 데이터는 삭제한다.

In [ ]:
included_id = list(jeju_poi['id'].unique())
add_id = list(jeju_poi_add['id'].unique())

for ID in add_id:
    if ID in included_id:
        jeju_poi_add = jeju_poi_add[jeju_poi_add['id'] != ID].copy()

- 120개 가량의 데이터가 삭제되었다.

In [ ]:
len(jeju_poi_add), len(add_id)

(1580, 1708)

In [ ]:
pd.DataFrame(jeju_poi_add['category_group_name'].value_counts())

,category_group_name
음식점,832
카페,514
관광명소,231
문화시설,3


In [ ]:
jeju_poi_add.to_excel('./data/220119/_서귀포_POI_최종RAW.xlsx', index=False)

## Selenium을 활용해 평점 & 이미지 데이터 크롤링

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome("driver/chromedriver") # 크롬 드라이버 경로 지정 
# driver.get("http://place.map.kakao.com/12710968") # Test : get 명령으로 접근하고 싶은 주소 지정  

### 제주시

In [ ]:
import pandas as pd
img_df = pd.read_excel("Final_pro/JEJU_POIAPI4.xlsx")
img_df.head()

In [ ]:
# 평점과 이미지 링크 컬럼을 추가
img_df["rating"] = ""
img_df["img_link"] = ""

In [ ]:
for idx,i in img_df[1558:].iterrows():
        time.sleep(0.7)
        driver = webdriver.Chrome("driver/chromedriver") # 크롬 드라이버 경로 지정 
        url = i["place_url"]
        driver.implicitly_wait(8)
        driver.get(url)
        try:
            contents = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b")
            
        except:
            img_df["rating"][idx] = "0.0"
            
        else:
            rating = contents.text
            img_df["rating"][idx] = rating

        try:
            image = driver.find_element_by_css_selector("#mArticle > div.cont_photo > div.photo_area > ul > li.size_l > a")
            
        except:
            name = str(idx) + "_" + i["place_name"]
            plt.savefig(f"Final_pro/error2/idx_{name}.png")
            driver.quit()

        else:
            img_link = image.get_attribute("style")[23:-3]
            img_link = "https:" + img_link
            img_df["img_link"][idx] = img_link

            response = requests.get(img_df["img_link"][idx])
            name = str(idx) + "_" + i["place_name"]
            with open("Final_pro/img2/{}.jpg".format(name), "wb") as f:
                f.write(response.content)
            driver.quit()
        

In [ ]:
img_df.to_excel("final_pro/merge_rating.xlsx")

#### 카테고리별 이미지 묶기

In [ ]:
import os

path = 'Final_pro/img'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

In [ ]:
import pandas as pd
place_xlsx = pd.read_excel("Final_pro/Have_img_place.xlsx").set_index("index")
place_xlsx.head()

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
index,,,,,,,,,,,,,,,
0,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874,33.247016,3.7,https://img1.kakaocdn.net/relay/local/R640x320...
1,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 914,AT4,관광명소,"여행 > 관광,명소 > 테마파크 > 워터테마파크",NaN,17150892,064-739-1930,제주워터월드,http://place.map.kakao.com/17150892,제주특별자치도 서귀포시 월드컵로 33,126.508546,33.245507,2.7,https://img1.kakaocdn.net/relay/local/R640x320...
2,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 토평동 1799,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,820826091,064-733-3500,윈드1947카트테마파크,http://place.map.kakao.com/820826091,제주특별자치도 서귀포시 토평공단로 78-27,126.588827,33.289519,3.7,https://img1.kakaocdn.net/relay/local/R640x320...
3,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 중문동 2565,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,780499262,NaN,액트몬 제주중문점,http://place.map.kakao.com/780499262,제주특별자치도 서귀포시 중문관광로 205,126.421517,33.243686,4.5,https://img1.kakaocdn.net/relay/local/R640x320...
4,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 색달동 2950-4,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10880885,1544-2988,퍼시픽 리솜 마린스테이지,http://place.map.kakao.com/10880885,제주특별자치도 서귀포시 중문관광로 154-17,126.415575,33.243856,2.9,https://img1.kakaocdn.net/relay/local/R640x320...


In [ ]:
import pandas as pd 
import shutil

for i in file_list_py:
    idx = int(i.split("_")[0])

    if place_xlsx["category_group_name"][idx]=="관광명소":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/관광명소/'
        shutil.move(src + filename, dir + filename)
    elif place_xlsx["category_group_name"][idx]=="카페":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/카페/'
        shutil.move(src + filename, dir + filename)
    elif place_xlsx["category_group_name"][idx]=="음식점":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/음식점/'
        shutil.move(src + filename, dir + filename)

### 서귀포시

In [ ]:
import pandas as pd
img_df2 = pd.read_excel("Final_pro/_서귀포_POI_최종RAW.xlsx")
img_df2.head()

In [ ]:
# 평점과 이미지 링크 컬럼을 추가
img_df2["rating"] = ""
img_df2["img_link"] = ""

In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
for idx,i in img_df2[1558:].iterrows():
        time.sleep(0.7)
        driver = webdriver.Chrome("driver/chromedriver") # 크롬 드라이버 경로 지정 
        url = i["place_url"]
        driver.implicitly_wait(8)
        driver.get(url)
        try:
            contents = driver.find_element_by_css_selector("#mArticle > div.cont_essential > div:nth-child(1) > div.place_details > div > div > a:nth-child(3) > span.color_b")
            
        except:
            img_df2["rating"][idx] = "0.0"
            
        else:
            rating = contents.text
            img_df2["rating"][idx] = rating

        try:
            image = driver.find_element_by_css_selector("#mArticle > div.cont_photo > div.photo_area > ul > li.size_l > a")
            
        except:
            name = str(idx) + "_" + i["place_name"]
            plt.savefig(f"Final_pro/error2/idx_{name}.png")
            driver.quit()

        else:
            img_link = image.get_attribute("style")[23:-3]
            img_link = "https:" + img_link
            img_df2["img_link"][idx] = img_link

            response = requests.get(img_df["img_link"][idx])
            name = str(idx) + "_" + i["place_name"]
            with open("Final_pro/img2/{}.jpg".format(name), "wb") as f:
                f.write(response.content)
            driver.quit()
        

C:\Users\alal8\AppData\Local\Temp/ipykernel_9352/3202358631.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img_df["rating"][idx] = rating
C:\Users\alal8\AppData\Local\Temp/ipykernel_9352/3202358631.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  img_df["img_link"][idx] = img_link


<Figure size 432x288 with 0 Axes>

In [ ]:
img_df2.to_excel("final_pro/merge_rating2.xlsx")

#### 카테고리별 이미지 묶기

In [ ]:
import os

path = 'Final_pro/img2'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.jpg')]

In [ ]:
import pandas as pd
place_xlsx = pd.read_excel("Final_pro/Have_img_place_seogipo.xlsx").set_index("index")
place_xlsx.head()

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,rating,img_link
index,,,,,,,,,,,,,,,
0,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 877-3,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10472331,064-739-8254,세리월드,http://place.map.kakao.com/10472331,제주특별자치도 서귀포시 법환상로2번길 97-17,126.511874,33.247016,3.7,https://img1.kakaocdn.net/relay/local/R640x320...
1,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 법환동 914,AT4,관광명소,"여행 > 관광,명소 > 테마파크 > 워터테마파크",NaN,17150892,064-739-1930,제주워터월드,http://place.map.kakao.com/17150892,제주특별자치도 서귀포시 월드컵로 33,126.508546,33.245507,2.7,https://img1.kakaocdn.net/relay/local/R640x320...
2,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 토평동 1799,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,820826091,064-733-3500,윈드1947카트테마파크,http://place.map.kakao.com/820826091,제주특별자치도 서귀포시 토평공단로 78-27,126.588827,33.289519,3.7,https://img1.kakaocdn.net/relay/local/R640x320...
3,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 중문동 2565,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,780499262,NaN,액트몬 제주중문점,http://place.map.kakao.com/780499262,제주특별자치도 서귀포시 중문관광로 205,126.421517,33.243686,4.5,https://img1.kakaocdn.net/relay/local/R640x320...
4,"테마파크,가볼만한 곳",제주특별자치도 서귀포시 색달동 2950-4,AT4,관광명소,"여행 > 관광,명소 > 테마파크",NaN,10880885,1544-2988,퍼시픽 리솜 마린스테이지,http://place.map.kakao.com/10880885,제주특별자치도 서귀포시 중문관광로 154-17,126.415575,33.243856,2.9,https://img1.kakaocdn.net/relay/local/R640x320...


In [ ]:
import pandas as pd 
import shutil

for i in file_list_py:
    idx = int(i.split("_")[0])

    if place_xlsx["category_group_name"][idx]=="관광명소":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/관광명소/'
        shutil.move(src + filename, dir + filename)
    elif place_xlsx["category_group_name"][idx]=="카페":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/카페/'
        shutil.move(src + filename, dir + filename)
    elif place_xlsx["category_group_name"][idx]=="음식점":
        filename = i
        src = 'Final_pro/img2/' 
        dir = 'Final_pro/img2/음식점/'
        shutil.move(src + filename, dir + filename)